In [2]:
import torch
from torch import nn
import torch.nn.functional as F

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
with open('/content/shakespeare.txt','r',encoding='utf8') as f:
    text = f.read()

In [10]:
(text[:1000])

"\n                     1\n  From fairest creatures we desire increase,\n  That thereby beauty's rose might never die,\n  But as the riper should by time decease,\n  His tender heir might bear his memory:\n  But thou contracted to thine own bright eyes,\n  Feed'st thy light's flame with self-substantial fuel,\n  Making a famine where abundance lies,\n  Thy self thy foe, to thy sweet self too cruel:\n  Thou that art now the world's fresh ornament,\n  And only herald to the gaudy spring,\n  Within thine own bud buriest thy content,\n  And tender churl mak'st waste in niggarding:\n    Pity the world, or else this glutton be,\n    To eat the world's due, by the grave and thee.\n\n\n                     2\n  When forty winters shall besiege thy brow,\n  And dig deep trenches in thy beauty's field,\n  Thy youth's proud livery so gazed on now,\n  Will be a tattered weed of small worth held:  \n  Then being asked, where all thy beauty lies,\n  Where all the treasure of thy lusty days;\n  To sa

In [11]:
print(text[:1000])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bud buriest thy content,
  And tender churl mak'st waste in niggarding:
    Pity the world, or else this glutton be,
    To eat the world's due, by the grave and thee.


                     2
  When forty winters shall besiege thy brow,
  And dig deep trenches in thy beauty's field,
  Thy youth's proud livery so gazed on now,
  Will be a tattered weed of small worth held:  
  Then being asked, where all thy beauty lies,
  Where all the treasure of thy lusty days;
  To say within thine own deep su

In [12]:
len(text)

5445609

# Encode text data

In [13]:
all_characters = set(text)

In [14]:
decoder = dict(enumerate(all_characters))

In [15]:
encoder = {char: ind for ind,char in decoder.items()}

In [16]:
encoded_text = np.array([encoder[char] for char in text])

In [17]:
encoded_text[:500]

array([ 7, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55, 55, 55, 33,  7, 55, 55, 54, 44, 67,  2, 55, 31, 28,  8,
       44, 25, 57, 34, 55, 46, 44, 25, 28, 34,  3, 44, 25, 57, 55, 74, 25,
       55, 75, 25, 57,  8, 44, 25, 55,  8, 21, 46, 44, 25, 28, 57, 25, 52,
        7, 55, 55, 82, 26, 28, 34, 55, 34, 26, 25, 44, 25,  9, 17, 55,  9,
       25, 28,  3, 34, 17, 69, 57, 55, 44, 67, 57, 25, 55,  2,  8, 56, 26,
       34, 55, 21, 25, 47, 25, 44, 55, 75,  8, 25, 52,  7, 55, 55, 37,  3,
       34, 55, 28, 57, 55, 34, 26, 25, 55, 44,  8, 15, 25, 44, 55, 57, 26,
       67,  3, 43, 75, 55,  9, 17, 55, 34,  8,  2, 25, 55, 75, 25, 46, 25,
       28, 57, 25, 52,  7, 55, 55, 36,  8, 57, 55, 34, 25, 21, 75, 25, 44,
       55, 26, 25,  8, 44, 55,  2,  8, 56, 26, 34, 55,  9, 25, 28, 44, 55,
       26,  8, 57, 55,  2, 25,  2, 67, 44, 17, 50,  7, 55, 55, 37,  3, 34,
       55, 34, 26, 67,  3, 55, 46, 67, 21, 34, 44, 28, 46, 34, 25, 75, 55,
       34, 67, 55, 34, 26

# one hot encoding

In [18]:
def one_hot_encoder(encoded_text, num_uni_chars):
  # Create a placeholder for zeros.
    one_hot = np.zeros((encoded_text.size, num_uni_chars))

    # Convert data type for later use with pytorch (errors if we dont!)
    one_hot = one_hot.astype(np.float32)

    # Using fancy indexing fill in the 1s at the correct index locations
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0


    # Reshape it so it matches the batch sahe
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))

    return one_hot

In [19]:
one_hot_encoder(np.array([1,2,0]),3)

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

# Creating Training Batches


In [20]:
example_text = np.arange(10)

In [21]:
# If we wanted 5 batches
example_text.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])

In [22]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):

    '''
    Generate (using yield) batches for training.

    X: Encoded Text of length seq_len
    Y: Encoded Text shifted by one

    Example:

    X:

    [[1 2 3]]

    Y:

    [[ 2 3 4]]

    encoded_text : Complete Encoded Text to make batches from
    batch_size : Number of samples per batch
    seq_len : Length of character sequence

    '''

    # Total number of characters per batch
    # Example: If samp_per_batch is 2 and seq_len is 50, then 100
    # characters come out per batch.
    char_per_batch = samp_per_batch * seq_len


    # Number of batches available to make
    # Use int() to roun to nearest integer
    num_batches_avail = int(len(encoded_text)/char_per_batch)

    # Cut off end of encoded_text that
    # won't fit evenly into a batch
    encoded_text = encoded_text[:num_batches_avail * char_per_batch]


    # Reshape text into rows the size of a batch
    encoded_text = encoded_text.reshape((samp_per_batch, -1))


    # Go through each row in array.
    for n in range(0, encoded_text.shape[1], seq_len):

        # Grab feature characters
        x = encoded_text[:, n:n+seq_len]

        # y is the target shifted over by 1
        y = np.zeros_like(x)

        #
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1]  = encoded_text[:, n+seq_len]

        # FOR POTENTIAL INDEXING ERROR AT THE END
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]

        yield x, y

In [23]:
sample_text = encoded_text[:20]

In [24]:
sample_text

array([ 7, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55,
       55, 55, 55])

In [25]:
batch_generator = generate_batches(sample_text,samp_per_batch=2,seq_len=5)

In [26]:
# Grab first batch
x, y = next(batch_generator)

In [27]:
x

array([[ 7, 55, 55, 55, 55],
       [55, 55, 55, 55, 55]])

In [28]:
y

array([[55, 55, 55, 55, 55],
       [55, 55, 55, 55, 55]])

In [29]:
torch.cuda.is_available()

True

In [30]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5,use_gpu=False):


        # SET UP ATTRIBUTES
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        #CHARACTER SET, ENCODER, and DECODER
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind,char in decoder.items()}


        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))


    def forward(self, x, hidden):


        lstm_output, hidden = self.lstm(x, hidden)


        drop_output = self.dropout(lstm_output)

        drop_output = drop_output.contiguous().view(-1, self.num_hidden)


        final_out = self.fc_linear(drop_output)


        return final_out, hidden


    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''

        if self.use_gpu:

            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))

        return hidden


# Creating the LSTM Model

In [31]:
class CharModel(nn.Module):

    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5,use_gpu=False):


        # SET UP ATTRIBUTES
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu

        #CHARACTER SET, ENCODER, and DECODER
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind,char in decoder.items()}


        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)

        self.dropout = nn.Dropout(drop_prob)

        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))


    def forward(self, x, hidden):


        lstm_output, hidden = self.lstm(x, hidden)


        drop_output = self.dropout(lstm_output)

        drop_output = drop_output.contiguous().view(-1, self.num_hidden)


        final_out = self.fc_linear(drop_output)


        return final_out, hidden


    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''

        if self.use_gpu:

            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))

        return hidden


# Instance of the Model

In [32]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=True,
)

In [33]:
total_param  = []
for p in model.parameters():
    total_param.append(int(p.numel()))

In [34]:
sum(total_param)

5470292

In [35]:
len(encoded_text)

5445609

Optimizer and Loss

In [36]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.CrossEntropyLoss()

In [37]:
# percentage of data to be used for training
train_percent = 0.9

In [38]:
len(encoded_text)

5445609

In [39]:
int(len(encoded_text) * (train_percent))

4901048

In [40]:
train_ind = int(len(encoded_text) * (train_percent))

In [41]:
train_data = encoded_text[:train_ind]
val_data = encoded_text[train_ind:]

# Training the Network
model parameters

In [42]:
## VARIABLES

# Epochs to train for
epochs = 50
# batch size
batch_size = 128

# Length of sequence
seq_len = 100

# for printing report purposes
# always start at 0
tracker = 0

# number of characters in text
num_char = max(encoded_text)+1

In [43]:
# Set model to train
model.train()


# Check to see if using GPU
if model.use_gpu:
    model.cuda()

for i in range(epochs):

    hidden = model.hidden_state(batch_size)


    for x,y in generate_batches(train_data,batch_size,seq_len):

        tracker += 1

        # One Hot Encode incoming data
        x = one_hot_encoder(x,num_char)

        # Convert Numpy Arrays to Tensor

        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)

        # Adjust for GPU if necessary

        if model.use_gpu:

            inputs = inputs.cuda()
            targets = targets.cuda()

        # Reset Hidden State
        # If we dont' reset we would backpropagate through all training history
        hidden = tuple([state.data for state in hidden])

        model.zero_grad()

        lstm_output, hidden = model.forward(inputs,hidden)
        loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())

        loss.backward()

        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
        nn.utils.clip_grad_norm_(model.parameters(),max_norm=5)

        optimizer.step()



        ###################################
        ### CHECK ON VALIDATION SET ######
        #################################

        if tracker % 25 == 0:

            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()

            for x,y in generate_batches(val_data,batch_size,seq_len):

                # One Hot Encode incoming data
                x = one_hot_encoder(x,num_char)


                # Convert Numpy Arrays to Tensor

                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                # Adjust for GPU if necessary

                if model.use_gpu:

                    inputs = inputs.cuda()
                    targets = targets.cuda()

                # Reset Hidden State
                # If we dont' reset we would backpropagate through
                # all training history
                val_hidden = tuple([state.data for state in val_hidden])

                lstm_output, val_hidden = model.forward(inputs,val_hidden)
                val_loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())

                val_losses.append(val_loss.item())

            # Reset to training model after val for loop
            model.train()

            print(f"Epoch: {i} Step: {tracker} Val Loss: {val_loss.item()}")

Epoch: 0 Step: 25 Val Loss: 3.2165865898132324
Epoch: 0 Step: 50 Val Loss: 3.2079741954803467
Epoch: 0 Step: 75 Val Loss: 3.197408437728882
Epoch: 0 Step: 100 Val Loss: 3.016463041305542
Epoch: 0 Step: 125 Val Loss: 2.890205144882202
Epoch: 0 Step: 150 Val Loss: 2.730475425720215
Epoch: 0 Step: 175 Val Loss: 2.6213436126708984
Epoch: 0 Step: 200 Val Loss: 2.496274471282959
Epoch: 0 Step: 225 Val Loss: 2.358293056488037
Epoch: 0 Step: 250 Val Loss: 2.2601816654205322
Epoch: 0 Step: 275 Val Loss: 2.18794846534729
Epoch: 0 Step: 300 Val Loss: 2.1369974613189697
Epoch: 0 Step: 325 Val Loss: 2.0799341201782227
Epoch: 0 Step: 350 Val Loss: 2.0396652221679688
Epoch: 0 Step: 375 Val Loss: 2.0066161155700684
Epoch: 1 Step: 400 Val Loss: 1.9725703001022339
Epoch: 1 Step: 425 Val Loss: 1.9368088245391846
Epoch: 1 Step: 450 Val Loss: 1.8988516330718994
Epoch: 1 Step: 475 Val Loss: 1.8748763799667358
Epoch: 1 Step: 500 Val Loss: 1.8434561491012573
Epoch: 1 Step: 525 Val Loss: 1.823357343673706
Epoc

In [44]:

model_name = 'example.net'

In [45]:
torch.save(model.state_dict(),model_name)

# Load Model

In [46]:
model = CharModel(
    all_chars=all_characters,
    num_hidden=512,
    num_layers=3,
    drop_prob=0.5,
    use_gpu=True,
)

In [47]:
model.load_state_dict(torch.load(model_name))
model.eval()

<ipython-input-47-35d2fda4df86>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_name))


CharModel(
  (lstm): LSTM(84, 512, num_layers=3, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc_linear): Linear(in_features=512, out_features=84, bias=True)
)

# Generating Predictions

In [48]:
def predict_next_char(model, char, hidden=None, k=1):

        # Encode raw letters with model
        encoded_text = model.encoder[char]

        # set as numpy array for one hot encoding
        # NOTE THE [[ ]] dimensions!!
        encoded_text = np.array([[encoded_text]])

        # One hot encoding
        encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))

        # Convert to Tensor
        inputs = torch.from_numpy(encoded_text)

        # Check for CPU
        if(model.use_gpu):
            inputs = inputs.cuda()


        # Grab hidden states
        hidden = tuple([state.data for state in hidden])


        # Run model and get predicted output
        lstm_out, hidden = model(inputs, hidden)


        # Convert lstm_out to probabilities
        probs = F.softmax(lstm_out, dim=1).data



        if(model.use_gpu):
            # move back to CPU to use with numpy
            probs = probs.cpu()


        # k determines how many characters to consider
        # for our probability choice.
        # https://pytorch.org/docs/stable/torch.html#torch.topk

        # Return k largest probabilities in tensor
        probs, index_positions = probs.topk(k)


        index_positions = index_positions.numpy().squeeze()

        # Create array of probabilities
        probs = probs.numpy().flatten()

        # Convert to probabilities per index
        probs = probs/probs.sum()

        # randomly choose a character based on probabilities
        char = np.random.choice(index_positions, p=probs)

        # return the encoded value of the predicted char and the hidden state
        return model.decoder[char], hidden

In [49]:
def generate_text(model, size, seed='The', k=1):



    # CHECK FOR GPU
    if(model.use_gpu):
        model.cuda()
    else:
        model.cpu()

    # Evaluation mode
    model.eval()

    # begin output from initial seed
    output_chars = [c for c in seed]

    # intiate hidden state
    hidden = model.hidden_state(1)

    # predict the next character for every character in seed
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)

    # add initial characters to output
    output_chars.append(char)

    # Now generate for size requested
    for i in range(size):

        # predict based off very last letter in output_chars
        char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)

        # add predicted character
        output_chars.append(char)

    # return string of predicted text
    return ''.join(output_chars)

In [50]:
print(generate_text(model, 1000, seed='The ', k=3))

The prophecies of two of the walls of the carping souls
    That the state of thy fool and souls are bloody
    That he hath bought thyself and me a man,
    To straight be best and truth than thou shalt see,
    And then, as to this charge, the seasons of him,
    Too store of such a mere than how I saw
    This foul displace the court of honour to his son,
    The tribunes and the state, and the more serves
    That she will be the state. If to be strange
    Were such a soul to think, that the bosom of
    The strength of men as the sun shall set her bed
    To shallow stirs to see the world of this.
    If thou hadst been thine eyes and there are strength
    As with his self-a service and all manner
    As this defence of hearts shall be an hour
    To stay a man to bring thee with my heart.
    Where's the more storm and to be treason fall'n?
    What will they be not?
  CLOTEN. With my deserving,
    The more, as I had not to see this stroke,
    The court and sour of him, and h

In [2]:
# torch.cuda.is_available()